In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [17]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import math

In [3]:
from src.SEC_crawler import SEC_crawler

In [4]:
after_date = '2007-01-01'
after_date = datetime.strptime(after_date, '%Y-%m-%d').date()

sec_crawler = SEC_crawler(after_date)

In [15]:
# Get table of the S&P 500 tickers, CIK, and industry from Wikipedia
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
cik_df = pd.read_html(wiki_url,header=0,index_col=0)[0]
cik_df['GICS Sector'] = cik_df['GICS Sector'].astype("category")
cik_df['GICS Sub Industry'] = cik_df['GICS Sector'].astype("category")
cik_df.head()

,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M Company,reports,Industrials,Industrials,"St. Paul, Minnesota",1976-08-09,66740,1902
ABT,Abbott Laboratories,reports,Health Care,Health Care,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie Inc.,reports,Health Care,Health Care,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABMD,ABIOMED Inc,reports,Health Care,Health Care,"Danvers, Massachusetts",2018-05-31,815094,1981
ACN,Accenture plc,reports,Information Technology,Information Technology,"Dublin, Ireland",2011-07-06,1467373,1989


In [16]:
df_company = cik_df[(cik_df['Date first added']<=str(after_date)) & (cik_df['GICS Sector'].isin(['Information Technology','Communication Services']))]

In [17]:
df_company

,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
ADBE,Adobe Inc.,reports,Information Technology,Information Technology,"San Jose, California",1997-05-05,796343,1982
GOOG,Alphabet Inc Class C,reports,Communication Services,Communication Services,"Mountain View, California",2006-04-03,1652044,1998
T,AT&T,reports,Communication Services,Communication Services,"Dallas, Texas",1983-11-30 (1957-03-04),732717,1983 (1885)
ADI,"Analog Devices, Inc.",reports,Information Technology,Information Technology,"Norwood, Massachusetts",1999-10-12,6281,1965
AAPL,Apple Inc.,reports,Information Technology,Information Technology,"Cupertino, California",1982-11-30,320193,1977
AMAT,Applied Materials Inc.,reports,Information Technology,Information Technology,"Santa Clara, California",1995-03-16,6951,1967
ADSK,Autodesk Inc.,reports,Information Technology,Information Technology,"San Rafael, California",1989-12-01,769397,NaN
ADP,Automatic Data Processing,reports,Information Technology,Information Technology,"Roseland, New Jersey",1981-03-31,8670,NaN
CTL,CenturyLink Inc,reports,Communication Services,Communication Services,"Monroe, Louisiana",1999-03-25,18926,NaN


In [18]:
%%time
df_list = []
company_list = df_company['CIK'].to_dict()
print(len(company_list))
for (ticker,cik) in tqdm(company_list.items()):
    df_list.append(sec_crawler.get_data(cik,ticker))
doc_links_df = pd.concat(df_list,axis=0)
doc_links_df = doc_links_df.set_index("ticker").join(cik_df['GICS Sector']).join(cik_df['GICS Sub Industry']).reset_index().rename(columns={"index":"ticker"})
doc_links_df.head()

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

30


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [38:15<00:00, 76.52s/it]

Wall time: 38min 15s


,ticker,cik,txt_link,doc_name,exhibit_link,item_no,accepted_date,period_of_report,GICS Sector,GICS Sub Industry
0,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-20-050884.txt,[],[Item 5.07],2020-02-27,2020-02-26,Information Technology,Information Technology
1,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-20-039203.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 7.01, Item 9.01]",2020-02-18,2020-02-17,Information Technology,Information Technology
2,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0000320193-20-000008.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 2.02, Item 9.01]",2020-01-28,2020-01-28,Information Technology,Information Technology
3,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-19-292676.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 8.01, Item 9.01]",2019-11-15,2019-11-07,Information Technology,Information Technology
4,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0000320193-19-000117.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 2.02, Item 9.01]",2019-10-30,2019-10-30,Information Technology,Information Technology


In [7]:
data_dir = '../data/Forms 8-K/'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [23]:
doc_links_df['exhibit_num'] = [len(i) for i in doc_links_df['exhibit_link']]

In [8]:
# doc_links_df.to_pickle(data_dir + 'doc_links_df.pkl')
doc_links_df = pd.read_pickle(data_dir + 'doc_links_df.pkl')

In [26]:
# doc_links_df.to_csv(data_dir + 'doc_links_df.csv', index=False)

In [9]:
pd.Series([len(i) for i in doc_links_df['exhibit_link']] ).value_counts()

1     2641
0     1410
2      727
3      290
4      135
5       57
6       22
7       18
8       12
10       7
11       3
9        2
12       2
13       1
29       1
16       1
14       1
50       1
21       1
dtype: int64

In [12]:
final_df = doc_links_df.loc[doc_links_df['exhibit_num']!=0].reset_index(drop=True)
print(len(final_df))
final_df.head()

3922


,ticker,cik,txt_link,doc_name,exhibit_link,item_no,accepted_date,period_of_report,GICS Sector,GICS Sub Industry,exhibit_num
0,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-20-039203.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 7.01, Item 9.01]",2020-02-18,2020-02-17,Information Technology,Information Technology,1
1,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0000320193-20-000008.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 2.02, Item 9.01]",2020-01-28,2020-01-28,Information Technology,Information Technology,1
2,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-19-292676.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 8.01, Item 9.01]",2019-11-15,2019-11-07,Information Technology,Information Technology,3
3,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0000320193-19-000117.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 2.02, Item 9.01]",2019-10-30,2019-10-30,Information Technology,Information Technology,1
4,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-19-242975.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 8.01, Item 9.01]",2019-09-11,2019-09-04,Information Technology,Information Technology,3


In [24]:
len(final_df)

3922

In [25]:
%%time
exhibit_doc_1 = final_df['exhibit_link'][:2000].apply(sec_crawler.extract_exhibit_doc_text)

Wall time: 37min 2s


In [26]:
%%time
exhibit_doc_2 = final_df['exhibit_link'][2000:].apply(sec_crawler.extract_exhibit_doc_text)

Wall time: 39min 56s


In [27]:
exhibit_doc_2

2000      EX-2.1 2 d935756dex21.htm EX-2.1   EX-2.1   ...
2001      EX-99.1 2 d926784dex991.htm EX-99.1   EX-99....
2002      EX-99.1 2 earningsreleaseq12015.htm EXHIBIT ...
2003      EX-99.1 2 reorgpressrelease462015.htm EXHIBI...
2004      EX-99.1 2 guidancepressrelease31215.htm EXHI...
                              ...                        
3917      EX-99.1 2 a07-7190_1ex99d1.htm EX-99.1      ...
3918      EX-99.1 2 a07-6829_1ex99d1.htm EX-99.1      ...
3919      EX-99.1 2 a07-6680_1ex99d1.htm EX-99.1      ...
3920      EX-99.1 2 a07-2136_1ex99d1.htm EX-99.1      ...
3921      EX-99.1 2 a07-1471_1ex99d1.htm EX-99        ...
Name: exhibit_link, Length: 1922, dtype: object

In [37]:
exhibit_doc = exhibit_doc_1.append(exhibit_doc_2)

In [38]:
final_df['exhibit_doc'] = exhibit_doc

In [44]:
final_df

,ticker,cik,txt_link,doc_name,exhibit_link,item_no,accepted_date,period_of_report,GICS Sector,GICS Sub Industry,exhibit_num,exhibit_doc
0,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-20-039203.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 7.01, Item 9.01]",2020-02-18,2020-02-17,Information Technology,Information Technology,1,EX-99.1 2 d845033dex991.htm EX-99.1 EX-99....
1,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0000320193-20-000008.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 2.02, Item 9.01]",2020-01-28,2020-01-28,Information Technology,Information Technology,1,EX-99.1 2 a8-kexhibit991q1202012.htm EXHIBIT...
2,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-19-292676.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 8.01, Item 9.01]",2019-11-15,2019-11-07,Information Technology,Information Technology,3,EX-1.1 2 d828969dex11.htm EX-1.1 EX-1.1 ...
3,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0000320193-19-000117.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 2.02, Item 9.01]",2019-10-30,2019-10-30,Information Technology,Information Technology,1,EX-99.1 2 a8-kexhibit991q4201992.htm EXHIBIT...
4,AAPL,320193,https://www.sec.gov/Archives/edgar/data/320193...,0001193125-19-242975.txt,[https://www.sec.gov/Archives/edgar/data/32019...,"[Item 8.01, Item 9.01]",2019-09-11,2019-09-04,Information Technology,Information Technology,3,EX-1.1 2 d777124dex11.htm EX-1.1 EX-1.1 ...
...,...,...,...,...,...,...,...,...,...,...,...,...
3917,XLNX,743988,https://www.sec.gov/Archives/edgar/data/743988...,0001104659-07-016308.txt,[https://www.sec.gov/Archives/edgar/data/74398...,"[Item 2.03, Item 3.02, Item 8.01, Item 9.01]",2007-03-05,2007-02-27,Information Technology,Information Technology,1,EX-99.1 2 a07-7190_1ex99d1.htm EX-99.1 ...
3918,XLNX,743988,https://www.sec.gov/Archives/edgar/data/743988...,0001104659-07-014817.txt,[https://www.sec.gov/Archives/edgar/data/74398...,"[Item 8.01, Item 9.01]",2007-02-28,2007-02-28,Information Technology,Information Technology,1,EX-99.1 2 a07-6829_1ex99d1.htm EX-99.1 ...
3919,XLNX,743988,https://www.sec.gov/Archives/edgar/data/743988...,0001104659-07-014386.txt,[https://www.sec.gov/Archives/edgar/data/74398...,"[Item 8.01, Item 9.01]",2007-02-27,2007-02-26,Information Technology,Information Technology,2,EX-99.1 2 a07-6680_1ex99d1.htm EX-99.1 ...
3920,XLNX,743988,https://www.sec.gov/Archives/edgar/data/743988...,0001104659-07-003171.txt,[https://www.sec.gov/Archives/edgar/data/74398...,"[Item 2.02, Item 9.01]",2007-01-18,2007-01-18,Information Technology,Information Technology,1,EX-99.1 2 a07-2136_1ex99d1.htm EX-99.1 ...


In [39]:
# final_df.to_pickle(data_dir + 'sec_8K_data.pkl')

In [43]:
# final_df.to_csv(data_dir + 'sec_8K_data.csv', index=False)